In [5]:
import time 
import pickle
from sklearn.model_selection import train_test_split

X = []
y = []
metadata = []

num_train = 1000000
num_test = 200000
current_time = time.strftime("%Y-%m-%d-%H%M")

X_RSA = None
y_RSA = None
metadata_RSA = None
    
with open("/workspace/lito9928/ASSEM/Nayeon/X_modified_40.pickle", "rb") as f:
    X_RSA = pickle.load(f)

with open("/workspace/lito9928/ASSEM/Nayeon/y_modified_40.pickle", "rb") as f:
    y_RSA = pickle.load(f)

with open("/workspace/lito9928/ASSEM/Nayeon/metadata_modified_40.pickle", "rb") as f:
    metadata_RSA = pickle.load(f)


X_RSA_train, X_RSA_test, y_RSA_train, y_RSA_test, metadata_RSA_train, metadata_RSA_test = train_test_split(
    X_RSA, y_RSA, metadata_RSA, test_size=0.2, shuffle=True, random_state=34
)
print(len(X_RSA_train), len(X_RSA_test))
'''
346896 86724
608528 152132
48336 12084
238096 59524
14544 3636
1280000 320000
525328 131332
1280000 320000
14800 3700
1280000 320000
'''

X_RSA_train = X_RSA_train
y_RSA_train = y_RSA_train
metadata_RSA_train = metadata_RSA_train
X_RSA_test = X_RSA_test
y_RSA_test = y_RSA_test
metadata_RSA_test = metadata_RSA_test

X = X_RSA_train + X_RSA_test
y = y_RSA_train + y_RSA_test
metadata = metadata_RSA_train + metadata_RSA_test

X_train, X_test, y_train, y_test, metadata_train, metadata_test = train_test_split(
    X, y, metadata, test_size=0.2, shuffle=True, random_state=34
)

1280000 320000


In [6]:
import numpy as np
import tensorflow as tf
print(tf.__version__)
tf.config.list_physical_devices('GPU')

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model,
        )

        # Compute sine and cosine components separately
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        # Interleave sine and cosine components to get the positional encoding
        pos_encoding = tf.stack([sines, cosines], axis=-1)
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])
        pos_encoding = pos_encoding[tf.newaxis, ...]

        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, : tf.shape(inputs)[1], :]


def scaled_dot_product_attention(query, key, value, mask):
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    if mask is not None:
        logits += mask * -1e9

    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output, attention_weights


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = (
            inputs["query"],
            inputs["key"],
            inputs["value"],
            inputs["mask"],
        )
        batch_size = tf.shape(query)[0]

        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))

        outputs = self.dense(concat_attention)

        return outputs


def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]


def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
    # print("padding mask enc:", padding_mask)
    attention = MultiHeadAttention(d_model, num_heads, name="attention")(
        {"query": inputs, "key": inputs, "value": inputs, "mask": padding_mask}
    )

    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=dff, activation="relu")(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)


def encoder(vocab_size, num_layers, dff, d_model, num_heads, dropout, name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")

    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            dff=dff,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)


def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)


def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    # print("look ahead mask dec:", look_ahead_mask)
    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(
        inputs={
            "query": inputs,
            "key": inputs,
            "value": inputs,
            "mask": look_ahead_mask,
        }
    )

    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)

    # print("padding mask dec:", padding_mask)
    attention2 = MultiHeadAttention(d_model, num_heads, name="attention_2")(
        inputs={
            "query": attention1,
            "key": enc_outputs,
            "value": enc_outputs,
            "mask": padding_mask,
        }
    )

    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(
        attention2 + attention1
    )

    outputs = tf.keras.layers.Dense(units=dff, activation="relu")(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name,
    )


def decoder(vocab_size, num_layers, dff, d_model, num_heads, dropout, name="decoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            dff=dff,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="decoder_layer_{}".format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name,
    )


def transformer(
    vocab_size, num_layers, dff, d_model, num_heads, dropout, name="transformer"
):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    print("inputs", inputs)

    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None), name="enc_padding_mask"
    )(inputs)

    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask, output_shape=(1, None, None), name="look_ahead_mask"
    )(dec_inputs)

    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None), name="dec_padding_mask"
    )(inputs)

    enc_outputs = encoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        dff=dff,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        dff=dff,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)


def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction="none"
    )(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)


class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=75):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(tf.cast(step, tf.float32))
        arg2 = tf.cast(step, tf.float32) * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2) 

with open("/workspace/lito9928/ASSEM/Seungheon/dataset/4-type-algorithm/RSA/opcode-only/RSA_opcode_only.txt", "r") as f:
    lines = f.readlines()
opcodes = set()
for line in lines:
    opcodes.add(line.rstrip())
with open("opcode_set.txt", "w") as file:
    for item in opcodes:
        file.write(item + "\n")


with open("opcode_set.txt", "r") as opcode_file:
    lines = opcode_file.readlines()

opcodes = list(map(lambda x: x.replace("\n", ""), lines))

op2idx = {word: i for i, word in enumerate(opcodes)}
idx2op = np.array(opcodes)

VOCAB_SIZE = len(opcodes)
START_TOKEN, END_TOKEN = VOCAB_SIZE, VOCAB_SIZE + 1

op2idx["START"] = VOCAB_SIZE
op2idx["END"] = VOCAB_SIZE + 1
idx2op = np.append(idx2op, ["START", "END"])

VOCAB_SIZE = VOCAB_SIZE + 2

import numpy as np

MAX_LENGTH = VOCAB_SIZE





2.6.2


In [7]:
# print(X[0])
# print(y[0])
# print(metadata[0])

import re 
def remove_condition_code_and_suffix(raw_sequence_str):
    matching_pattern = None
    global non_suffix_instruction_list
    for pattern in non_suffix_instruction_list:
        if re.match(pattern, raw_sequence_str):
            matching_pattern = pattern
            break
    if matching_pattern:
        return matching_pattern
    else:
        return raw_sequence_str

BATCH_SIZE = 256

print("start spliting")

def encode(code):
    
    return list(map(lambda x: op2idx[(x)], code))


def decode(idx):
    return list(map(lambda x: idx2op[(x)], idx))


errors = tf.keras.preprocessing.sequence.pad_sequences(
    [
        np.append(np.insert(encode(x.split()), 0, op2idx["START"]), op2idx["END"])
        for x in X_train
    ],
    maxlen=MAX_LENGTH,
    padding="post",
)

corrections = tf.keras.preprocessing.sequence.pad_sequences(
    [
        np.append(np.insert(encode(x.split()), 0, op2idx["START"]), op2idx["END"])
        for x in y_train
    ],
    maxlen=MAX_LENGTH,
    padding="post",
)
dec_inputs = []
outputs = []

for correction in corrections:
    dec_inputs.append(correction[:-1])
    outputs.append(correction[1:])


print("start making dataset")

dataset = tf.data.Dataset.from_tensor_slices(
    (
        {"inputs": errors, "dec_inputs": dec_inputs},
        {"outputs": outputs},
    )
)

print("dataset done")

dataset = dataset.cache()
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

DATASET_SIZE = len(list(dataset))


start spliting


start making dataset
dataset done


In [8]:
print(len(X_train))
print(len(X_test))

1280000
320000


In [9]:
errors[0]

array([82, 64, 68, 35, 74, 53, 38, 60, 24, 53, 50, 83,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [10]:
y_train[0]

'adds ldr cmp bhi.n add.w ldr.w umull adds.w add.w str.w'

In [11]:
gpus = tf.config.experimental.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
print('\n\n Running on multiple GPUs ', [gpu.name for gpu in gpus])
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# import os

# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]= "1"

# import torch
# import torch.nn as nn

# # device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
# # # torch.device("cuda" if torch.cuda.is_available() else "cpu")

# GPU_NUM = 5 # 원하는 GPU 번호 입력
# device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(device) # change allocation of current GPU


# print('Device:', device)  # 출력결과: cuda 
# print('Count of using GPUs:', torch.cuda.device_count())   
# print('Current cuda device:', torch.cuda.current_device()) 

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


 Running on multiple GPUs  ['/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


In [12]:
with strategy.scope():
    # training
    # tf.keras.backend.clear_session()

    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.compat.v1.Session(config=config)

    print(np.__version__)

    NUM_LAYERS = 2
    D_MODEL = 48
    NUM_HEADS = 16
    DFF = 16
    DROPOUT = 0.1

    model = transformer(
        vocab_size=VOCAB_SIZE,
        num_layers=NUM_LAYERS,
        dff=DFF,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
    )

    learning_rate = CustomSchedule(d_model=D_MODEL)

    optimizer = tf.keras.optimizers.Adam(
        learning_rate=learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
    )


    def accuracy(y_true, y_pred):
        y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


    print("start compiling model")

    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])


print("start fit")
EPOCHS = 10
model.fit(dataset, epochs=EPOCHS, verbose=1)
# batch = 1
# epoch - 10

# model.save("assem_transformer.h5")


1.19.2
inputs KerasTensor(type_spec=TensorSpec(shape=(None, None), dtype=tf.float32, name='inputs'), name='inputs', description="created by layer 'inputs'")
start compiling model
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
start fit
Epoch 1/10
INFO:tensorflow:batch_all_reduce: 86 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/

In [13]:
model.save_weights('transformer_modified_RSA_1021')

In [14]:
with open("X_test.pickle","wb") as fw:
    pickle.dump(X_test, fw)

with open("y_test.pickle","wb") as fw:
    pickle.dump(y_test, fw)
    
with open("meta_test.pickle", "wb") as fw:
    pickle.dump(metadata_test, fw)

: 